In [ ]:
from src import common
import pandas as pd

## Load original FAQ

In [ ]:
## Original FAQ
df = pd.read_csv('Original_FAQ_M365.csv')

In [ ]:
df.head()

In [ ]:
## COnfiguration
tools = common.AOAI_TOOLS(config_file='./config.yml')

In [ ]:
## Load prompts to be used
tools.load_prompts(prompt_name='QA_prompt_template', 
                    prompt_path='./prompts/operational_prompts.yml')

## Configuration for AOAI model
tools.setClient()

## Augment Questions

In [ ]:
## load prompts
prompt_template = tools.promptBank['QA_prompt_template']['PROMPTS']['Augment_Q_from_QA_template']
prompt_template

In [ ]:
## Augument Questions with original questions and the combination of questions and answers
df_all = tools.generate_df_for_augmented_Questions(df=df,
                                                    prompt_template=prompt_template,
                                                    need_answer=True,)

In [ ]:
## Vectorize the augmented questions
#df_all['search_vector'] = df_all['augmented_query'].apply(lambda x: tools.getEmbeddedVector(text=x))

In [ ]:
## Save the datasets
#df_all.to_pickle('df_all.pkl')

In [ ]:
df_all = pd.read_pickle('df_all.pkl')
df_all.head()

## Quick validation

In [ ]:
## load prompts
prompt_template = tools.promptBank['QA_prompt_template']['PROMPTS']['Augment_Q_from_Q_template']
prompt_template

In [ ]:
## 元の質問、もしくは質問＋回答を入力として、質問を増幅する
#df_queries = tools.generate_df_for_augmented_Questions(df=df,
#                                                    prompt_template=prompt_template,
#                                                    need_answer=False,)

In [ ]:
## Vectorize the augmented questions
#df_queries['search_vector'] = df_queries['augmented_query'].apply(lambda x: tools.getEmbeddedVector(text=x))

In [ ]:
#df_queries.to_pickle('df_queries.pkl')
df_queries = pd.read_pickle('df_queries.pkl')

In [ ]:
df_queries.head()

## Quick evaluation

In [ ]:
## Validation
df_evaluation = tools.generate_df_evaluation(df_all=df_all, df_queries=df_queries)

In [ ]:
df_evaluation['correctness'].sum() / len(df_evaluation)

In [ ]:
df_evaluation.head()

## manual conversation

In [ ]:
## load prompts
prompt_template = tools.promptBank['QA_prompt_template']['PROMPTS']
prompt_template

In [ ]:
INPUT = 'こんにちは'

In [ ]:
query_result = tools.judge_if_KB(INPUT=INPUT,
                            df_all=df_all)
prompt = tools.set_prompt(prompt_template=prompt_template,
                          query_result=query_result,
                          threshold=0.6)

In [ ]:
conversation_history = []
conversation_history, returned_message = tools.manualConversation(INPUT=INPUT,
                                            prompt=prompt,
                                            conversation_history=conversation_history,)

In [ ]:
INPUT = 'Microsoft 365 サブスクリプションを10人で共有できると聞いたのですが？'
conversation_history = tools.manualConversation(INPUT=INPUT,
                         prompt=prompt,
                         conversation_history=conversation_history,)

In [ ]:
conversation_history